In [8]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1500)

warnings.filterwarnings('ignore')
np.random.seed(1)

In [21]:
def xgboost_lr_train_test(File):
    start = time.clock()
    train_df, test_df = train_test_split(File, train_size = 0.8)
    X_train = train_df.drop(train_df.columns[0], axis=1)
    y_train = train_df[train_df.columns[0]]
    X_test = test_df.drop(test_df.columns[0], axis=1)
    y_test = test_df[test_df.columns[0]]
    
    #training xgboost
    xgbclf = xgb.XGBClassifier(nthread=4, learning_rate=0.1,
                            n_estimators=30, max_depth=7,  subsample=0.9, colsample_bytree=0.5)
    xgbclf.fit(X_train, y_train)
    
    #using xgboost to encode train set and test set features
    X_train_leaves = xgbclf.apply(X_train)
    train_rows = X_train_leaves.shape[0]
    xgbclf.fit(X_test, y_test)
    X_test_leaves = xgbclf.apply(X_test)
    
    
    #fittting lr using just xgboost encoded feature
    lr = LogisticRegression(n_jobs=-1, C=0.1, penalty='l1')
    lr.fit(X_train_leaves,y_train)
    
    y_pred = lr.predict(X_test_leaves)
    gbdtlr_auc1 = roc_auc_score(y_test, y_pred)
    print('XGBDT+LR AUC : %.5f' % gbdtlr_auc1)
    
    
    X_train_ext = hstack([X_train_leaves, X_train])
    lr.fit(X_train_ext, y_train)
    X_test_ext = hstack([X_test_leaves, X_test])
    
    y_pred_2=lr.predict(X_test_ext)
    gbdtlr_auc2 = roc_auc_score(y_test, y_pred_2)
    print('XGBDT+LR AUC2 : %.5f' % gbdtlr_auc2)
    
    


In [22]:
example = pd.read_csv('example.csv')

In [23]:
xgboost_lr_train_test(example)

XGBDT+LR AUC : 0.49181
XGBDT+LR AUC2 : 0.54227
